In [25]:
import pandas as pd
import re
import json
from datetime import datetime

# Sample JSON data
sample_json = '''
[
    {
        "productId": "57499220",
        "productName": "AMD 라데온 PRO W7900 Dual Slot D6 48GB",
        "productPriceStr": "가격비교예정",
        "productSpec": "W7900 / GCD 5nm, MCD 6nm / 스트림 프로세서: 6144개",
        "productImage": "https://img.danawa.com/prod_img/500000/220/499/img/57499220_1.jpg?shrink=130:130&_v=20240619161118",
        "productUrlAll": "https://prod.danawa.com/info/?pcode=57499220&cate=112753"
    },
    {
        "productId": "57483869",
        "productName": "COLORFUL 지포스 RTX 4070 토마호크 EX D6X 12GB 피씨디렉트",
        "productPriceStr": "798,000",
        "productSpec": "RTX 4070 / 4nm / 베이스클럭: 1920MHz / 부스트클럭: 2475MHz / 스트림 프로세서: 5888개",
        "productImage": "//img.danawa.com/prod_img/500000/869/483/img/57483869_1.jpg?shrink=130:130&_v=20240619111211",
        "productUrlAll": "https://prod.danawa.com/info/?pcode=57483869&cate=112753"
    },
    {
        "productId": "53521316",
        "productName": "COLORFUL iGame 지포스 RTX 4060 Ti 용 Edition OC D6 8GB 웨이코스",
        "productPriceStr": "612,140",
        "productSpec": "RTX 4060 Ti / 4nm / 베이스클럭: 2310MHz / 부스트클럭: 2580MHz / 스트림 프로세서: 4352개 / PCIe4.0x16(at x8)",
        "productImage": "//img.danawa.com/prod_img/500000/316/521/img/53521316_1.jpg?shrink=130:130&_v=20240514101837",
        "productUrlAll": "https://prod.danawa.com/info/?pcode=53521316&cate=112753"
    }
]
'''

# Load JSON data
data = json.loads(sample_json)

# Convert JSON data to DataFrame
danawa_gpu = pd.DataFrame(data)

# List of words to be removed from the model
words_to_remove = ['대원시티에스', '피씨디렉트', '디앤디컴', '제이씨현', '에즈윈', '대원씨티에스', '웨이코스', '마이크로닉스', '지포스', '라데온', 'D6X']

# Color translation dictionary
color_translation = {
    '핑크': 'Pink', '화이트': 'White', '블랙': 'Black', '레드': 'Red', 
    '블루': 'Blue', '그린': 'Green', '옐로우': 'Yellow', '퍼플': 'Purple', 
    '실버': 'Silver', '골드': 'Gold'
}

# Add chipset_type column based on productName containing '라데온'
danawa_gpu['chipset_type'] = danawa_gpu['productName'].apply(lambda x: 'AMD' if '라데온' in x else 'NVIDIA')
danawa_gpu['Type'] = "GPU"

# Function to extract company, model, memory, color, and RGB information
def extract_company_model_memory_color_rgb(row):
    parts = row.split()
    company = parts.pop(0)  # First word is the company
    memory = ""
    color = ""
    model_parts = []
    has_rgb = False

    for part in parts:
        if re.search(r'\d+GB', part):  # Check if part contains memory information
            memory = part.replace('GB', '').strip()
        elif part.lower() in color_translation.keys() or part.lower() in [v.lower() for v in color_translation.values()]:
            color = color_translation.get(part.lower(), part.capitalize())
        elif part.upper() == 'RGB':
            has_rgb = True
        elif part not in words_to_remove:
            model_parts.append(part)
    
    model = " ".join(model_parts).strip().upper()  # Remaining parts form the model name

    return company, model, memory, color, has_rgb

# Apply the function to the productName column
danawa_gpu[['Company', 'Model', 'Memory', 'Color', 'RGB']] = danawa_gpu['productName'].apply(lambda x: pd.Series(extract_company_model_memory_color_rgb(x)))

# Function to clean the productName and remove parentheses and RGB
def clean_product_name_v2(product_name):
    cleaned_name = re.sub(r'\(.*?\)', '', product_name)  # Remove parentheses
    cleaned_name = re.sub(r'\d+GB', '', cleaned_name)  # Remove memory information
    cleaned_name = re.sub(r'\b(black|white|red|blue|green|yellow|purple|silver|gold|핑크|화이트|블랙|레드|블루|그린|옐로우|퍼플|실버|골드)\b', '', cleaned_name, flags=re.IGNORECASE)  # Remove color information
    cleaned_name = re.sub(r'\bRGB\b', '', cleaned_name, flags=re.IGNORECASE)  # Remove RGB
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip().upper()  # Remove extra spaces and convert to uppercase
    return cleaned_name

# Apply the function to clean the productName column
danawa_gpu['productName'] = danawa_gpu['productName'].apply(clean_product_name_v2)

# Remove rows where productName contains '해외'
danawa_gpu = danawa_gpu[~danawa_gpu['productName'].str.contains('해외')]

# Remove commas from productPriceStr column and rename it to Price
danawa_gpu['Price'] = danawa_gpu['productPriceStr'].str.replace(',', '')
danawa_gpu.drop(columns=['productPriceStr'], inplace=True)

# Add ComponentID column
danawa_gpu['ComponentID'] = danawa_gpu.apply(lambda x: f"{x['Type']}#{x['Company']}#{x['Model']}", axis=1)

# Add Date column with current date and time
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
danawa_gpu['Date'] = current_time

# Add Shop column
danawa_gpu['Shop'] = 'danawa'

# Rename productUrlAll to URL
danawa_gpu.rename(columns={'productUrlAll': 'URL'}, inplace=True)

# Rename productImage to image_url and add https: if it starts with //
danawa_gpu.rename(columns={'productImage': 'image_url'}, inplace=True)
danawa_gpu['image_url'] = danawa_gpu['image_url'].apply(lambda x: 'https:' + x if x.startswith('//') else x)

# Convert RGB column to boolean
danawa_gpu['RGB'] = danawa_gpu['RGB'].astype(bool)

# 중복 데이터 제거
# 부품구분_제조사_모델명
danawa_gpu.drop_duplicates(subset=['ComponentID', 'URL'], inplace=True)

# Drop productId, productName, and productSpec columns
danawa_gpu.drop(columns=['productId', 'productName', 'productSpec'], inplace=True)

# Load the standard GPU CSV data
standard_gpu_path = 'gpu_standard.csv'
standard_gpu = pd.read_csv(standard_gpu_path)

# Create ComponentID in standard_gpu
standard_gpu['ComponentID'] = 'GPU#' + standard_gpu['회사'] + '#' + standard_gpu['모델']

# Convert 메모리 column to int and then to string
standard_gpu['메모리'] = standard_gpu['메모리'].astype(int).astype(str)

# Join danawa_gpu and standard_gpu on ComponentID and Memory
final_gpu = pd.merge(danawa_gpu, standard_gpu, left_on=['ComponentID', 'Memory'], right_on=['ComponentID', '메모리'], how='inner')

# Drop the columns that are no longer needed after join
final_gpu.drop(columns=['메모리', '회사', '모델'], inplace=True)
final_gpu.fillna('', inplace = True)

# Convert final DataFrame back to JSON
final_data = final_gpu[['ComponentID', 'Type', 'Date', 'Shop', 'Price', 'URL']].copy()
final_json_data = final_data.to_dict(orient='records')

# Save the final cleaned JSON data
final_output_path_v2 = 'processed_danawa_gpu2222.json'
with open(final_output_path_v2, 'w', encoding='utf-8') as f:
    json.dump(final_json_data, f, ensure_ascii=False, indent=4)

# Display the first few rows of the updated dataframe
print(final_gpu.head())


                                           image_url  \
0  https://img.danawa.com/prod_img/500000/869/483...   

                                                 URL chipset_type Type  \
0  https://prod.danawa.com/info/?pcode=57483869&c...       NVIDIA  GPU   

    Company             Model Memory Color  RGB_x   Price  ...     길이   기본전력  \
0  COLORFUL  RTX 4070 토마호크 EX     12        False  798000  ...  309.0  200.0   

    최대전력   전원부       코어온도           소음  3DMark TS  드라이버 버전  색상  RGB_y  
0  200.0  10.0  64.9 (25)  42.7 (17.6)     17,962   531.42      False  

[1 rows x 27 columns]
